Warning: This notebook might not work because it requires an OpenAI API key

In [ ]:
import os
os.chdir('..')

from typing import List
from pathlib import Path

from dotenv import load_dotenv
from pydantic import BaseModel
from openai import OpenAI

from src.scripts.submit_openai_batch_job import system_prompt, get_user_prompt, get_movie_ids, format_user_prompt

In [ ]:
load_dotenv() # set the OPENAI_API_KEY environment variable in your .env file (or in your shell)
client = OpenAI()
input_dir = Path("./data/interim/")
print(input_dir.is_dir())
print("Metadata files:", len(list(input_dir.glob("character.metadata_*.csv"))))
print("Plot summaries:", len(list(input_dir.glob("plot_summaries_*.txt"))))

In [9]:
movie_ids = get_movie_ids(input_dir)
print("Movie IDs in common:", len(movie_ids))

Movie IDs in common: 37781


In [10]:
# define output objects
class Character(BaseModel):
    name: str
    dies: bool

class Characters(BaseModel):
    characters: List[Character]

In [13]:
# test with a few movies and edge cases
N = 5
movie_ids = get_movie_ids(input_dir)
movie_ids = movie_ids[:N] + ["4730"] # add the Batman example for comparison

movie_prompts = [get_user_prompt(movie_id, input_dir) for movie_id in movie_ids]

movie_prompts = [prompt for prompt in movie_prompts if prompt] # remove empty prompts (no character names)

test_cases = [
    (["Alice", "Bob"], "Alice and Bob are going to a science fair on Friday."),
    (["Alice", "Bob"], "The solar system is large, but earth has only 1 moon."),
    (["Alice", "Bob"], "Bob drops a bomb on Alice."),
    (["Alice", "Bob"], "Carol dies"),
    (["Alice", "Bob"], "Alice and Bob were on board Malaysia Airlines Flight 370."),
    (["Alice", "Bob"], "dies death killed murdered drowned"),
    (["Gus Grissom", "Gus Grissom Jr.", "Gus Grissom Sr.", "Virgil Ivan Grissom"], "Gus was the command pilot for Apollo 1."),
]

test_prompts = [format_user_prompt(characters, plot_summary) for characters, plot_summary in test_cases]

user_prompts = movie_prompts + test_prompts

for i, user_prompt in enumerate(user_prompts):
    print(i, user_prompt)

0 Character names: Viola
Plot summary: Three friends: Max , Samuele  and Nicola  goes on holiday with their caravan. During the trip, however, Viola , who has just robbed a bank, can get on the campers and takes them hostage. Initially, the three are frightened, but gradually gain confidence with the girl and they fall in love. In the end they take the decision to help her in the delivery of some coins of inestimable value.
1 Character names: Prince Edward
Plot summary: Edward, Prince of Wales, son and heir to his father King Edward III of England, leads an English army to the French province of Aquitaine to protect the inhabitant from the ravages of the French. After defeating the French in battle, the defeated French plot to kill the prince. Failing in this, they kidnap his lady, the lovely Lady Joan Holland. Of course Prince Edward has to ride to the rescue, adopting numerous guises to save his paramour, which ultimately end in him leading his men into one final climactic battle aga

In [14]:
for i, user_prompt in enumerate(user_prompts):

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format=Characters,
    )

    event = completion.choices[0].message.parsed

    print(i, event)

0 characters=[Character(name='Viola', dies=False), Character(name='Max', dies=False), Character(name='Samuele', dies=False), Character(name='Nicola', dies=False)]
1 characters=[Character(name='Prince Edward', dies=False)]
2 characters=[Character(name='Pratap Singh', dies=False), Character(name="Kalicharan's wife", dies=False)]
3 characters=[Character(name='Batman', dies=False), Character(name='Harvey Dent', dies=True), Character(name='Dr. Chase Meridian', dies=False), Character(name='Sugar', dies=False), Character(name='James Gordon', dies=False), Character(name='Dr. Burton', dies=False), Character(name='Spice', dies=False), Character(name='Alfred Pennyworth', dies=False), Character(name='Riddler', dies=False), Character(name='Robin', dies=False)]
4 characters=[Character(name='Alice', dies=False), Character(name='Bob', dies=False)]
5 characters=[Character(name='Alice', dies=False), Character(name='Bob', dies=False)]
6 characters=[Character(name='Alice', dies=True), Character(name='Bob'

The model infers:
```
Unnamed characters (0 and 7)
Harvey Dent is Two-Face (3), so he dies
Dropping a bomb on someone is likely to result in their death (6)
Everyone on MH370 died (8)
Gus Grissom's legal name is Virgil Ivan Grissom (10)
```

Test case 9 fails, the model does not separate `Character names` from `Plot summary`. We should consider adding examples to the message chain.